# Setup

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from jupyter_bbox_widget import BBoxWidget
from plotly.express import imshow as imshow_interactive

In [ ]:
import sys
import pathlib

REPO_DIR = pathlib.Path(__name__).resolve().parent.parent
SRC_DIR = str(REPO_DIR / "src")
sys.path.append(SRC_DIR)

In [ ]:
from scanplot.io import read_image_rgb, dump_coords_csv
from scanplot.plotting import draw_image, draw_ROI
from scanplot.view import CoordinatesMapper, DetectorWidget

from scanplot.core import (
    template_match,
    replace_black_pixels,
    normalize_map,
    image_tresholding,
    reconstruct_template_mask,
    generalized_hough_transform,
    CoordinatesConverter,
    bboxes_to_roi,
    apply_roi,
)

# Algorithm Pipeline

## 1. Upload plot image and template image

Input data examples can be found in `<REPOSITORY>/datasets/`.

In [ ]:
DATA_PATH = pathlib.Path("../datasets/")

In [ ]:
PLOT_NUMBER = 51
MARKER_NUMBER = 1

PLOT_PATH = DATA_PATH / "plot_images" / f"plot{PLOT_NUMBER}.png"
TEMPLATE_PATH = (
    DATA_PATH / "marker_images" / f"plot{PLOT_NUMBER}_marker{MARKER_NUMBER}.png"
)

In [ ]:
source_image = read_image_rgb(PLOT_PATH)
source_template = read_image_rgb(TEMPLATE_PATH)

image = replace_black_pixels(source_image, value=10)
template = replace_black_pixels(source_template, value=10)

In [ ]:
plt.subplot(1, 2, 1)
draw_image(image)
plt.title("Source image")

plt.subplot(1, 2, 2)
draw_image(template)
plt.title("Chosen marker")

## 2. (optional step) Select region of interest

Default ROI is a whole image

In [ ]:
roi_widget = BBoxWidget(
    hide_buttons=True,
    classes=["Region of interest"],
    image_bytes=cv.imencode(".png", source_image)[1].tobytes(),
    colors=["green"],
)

In [ ]:
display(roi_widget)

In [ ]:
roi = bboxes_to_roi(source_image, roi_widget.bboxes)

draw_image(source_image)
draw_ROI(roi)

In [ ]:
image = apply_roi(image, roi)

## 3. Run matching algorithms

3.1. Perform template tresholding and create template masks

In [ ]:
template_mask = image_tresholding(source_template, treshold=100)
additional_template_mask = reconstruct_template_mask(template_mask)

# fig = plt.figure()
# plt.subplot(1, 2, 1)
# draw_image(template_mask)

# plt.subplot(1, 2, 2)
# draw_image(additional_template_mask)

In [ ]:
# draw_image(template_mask)

3.2. Run template matching algorithm and compute correlatoin map

In [ ]:
correlation_map, _ = template_match(
    image, template, template_mask, norm_result=True
)

correlation_map2, _ = template_match(
    image, template, template_mask, norm_result=True,
    method_name="cv.TM_SQDIFF"
)

correlation_map_additional, _ = template_match(
    image, template, additional_template_mask, norm_result=True
)

3.3. Run Hough transform algorithm and compute accumulator array

In [ ]:
accumulator = generalized_hough_transform(
    image, template, norm_result=True, crop_result=True
)

assert correlation_map.shape == accumulator.shape

3.4. Combine correlation map obtained by template matching algorithm and accumulator array from Hough Transform

In [ ]:
# correlation_map_combined = correlation_map + 0.7 * correlation_map_additional
# correlation_map_combined = normalize_map(correlation_map_combined)

# correlation_map_with_hough = correlation_map + 0.6 * accumulator
# correlation_map_with_hough = normalize_map(correlation_map_with_hough)

# correlation_map_new = (correlation_map + correlation_map2) / 2
# correlation_map_with_hough = correlation_map_new + 0.6 * accumulator

correlation_map_with_hough = correlation_map + 0.6 * accumulator
correlation_map_with_hough = normalize_map(correlation_map_with_hough)

In [ ]:
imshow_interactive(correlation_map_with_hough)

## 4. Select algorithm parameters

The algorithm has 2 parameters:
- Points Number
- Points Density

Learn more about parameters selection in [documentation](https://github.com/adusachev/scanplot/blob/develop/docs/user_manual.md#algorithm-parameters).

In [ ]:
detector = DetectorWidget(
    source_image=source_image,
    template=template,
    correlation_map=correlation_map_with_hough
)

In [ ]:
widget_settings = {
    "fig_size": 9,
    "marker_size": 50,
    "marker_color": "yellow",
    "marker_type": "*",
}

detector_widget = detector.main_widget(**widget_settings)
display(detector_widget)

In [ ]:
detected_points_px = detector.get_detections()

# points coordinates in pixels (!)
x = detected_points_px[:, 0]
y = detected_points_px[:, 1]
plt.scatter(x, y)

## 5. Convert obtained coordinates from pixel to real values

Map pixel coordinates to factual coordinates

In [ ]:
mapper = CoordinatesMapper(source_image)

In [ ]:
fig_size = 10

mapper_widget = mapper.interactive_widget(fig_size=fig_size)
display(mapper_widget)

Convert pixel coordinates to factual coordinates

In [ ]:
converter = CoordinatesConverter()
converter.import_parameters_from_mapper(mapper)

In [ ]:
x_px = detected_points_px[:, 0]
y_px = detected_points_px[:, 1]

x_factual, y_factual = converter.from_pixel(x_pixel=x_px, y_pixel=y_px)

In [ ]:
plt.scatter(x_factual, y_factual)

Save obtained coordinates in csv

In [ ]:
dump_coords_csv(
    x=x_factual,
    y=y_factual,
    savepath=f"detections_plot{PLOT_NUMBER}_marker{MARKER_NUMBER}.csv"
)